## LIME example: simple binary text classifier
For this example, we use a pre-trained model used to predict the sentiment (positive
or negative) of IMDB movie reviews. The tutorial for creating that model is linked in the GitHub repo's README.

Requirements for successfully running this notebook are numpy, tensorflow, and of course, LIME.
### 0. Imports

In [1]:
import pickle

import numpy as np
import tensorflow as tf
from lime.lime_text import LimeTextExplainer

### 1. Class definition
The following class is a straightforward wrapper around the TensorFlow model's prediction
function, in order to correctly interface with the LIME API.

In [ ]:
class KerasClassifier:
    loaded_model = tf.keras.models.load_model('model.h5')
    with open('tokenizer.pickle', 'rb') as f:
        loaded_tokenizer = pickle.load(f)

    def prepare_data(self, strings):
        sequences = self.loaded_tokenizer.texts_to_sequences(strings)
        sequences = tf.keras.preprocessing.sequence.pad_sequences(sequences, maxlen=128, padding="post")
        return sequences

    def get_probabilites(self, strings):
        sequences = self.prepare_data(strings=strings)
        predictions = self.loaded_model.predict([sequences])

        probability_list = []
        for prediction in predictions:
            probability_raw = prediction[0]
            probability_list.append([probability_raw, (1 - probability_raw)])

        return np.array(probability_list)



### 2. Getting some movie reviews


In [ ]:
review_joker = "Truly a masterpiece, The Best Hollywood film of 2019, one of the Best films of the decade... And truly the "\
          "Best film to bring a comic book so chillingly and realistically to real ife. Remarkable Direction, " \
          "Cinematography, Music and the Acting. Some people are surprised to find it DISTURBING and VIOLENT, " \
          "but it's a necessity and message. It's about society and reflects those " \
          "underappreciated/unrecognized/bullied people, proving they can do something too. The way it shows class " \
          "difference, corruption and how rich and talented rule others around them is not exaggerated and that's " \
          "what makes it different. It's BELIEVABLE. There could be multiple JOKERs living in our society that could " \
          "shake those around them in much bitter way than the film shows making people uncomforting people. Consider " \
          "this a wake up call, a message, but first a film. A PERFECT film. "

review_cats = "This movie reminds me of that scene from Jurassic Park where Jeff Goldblum says 'You were so preoccupied " \
          "with whether or not you could, you never bothered to ask if you should.' This was hands down the most " \
          "disturbingly awful movie I have ever seen. Whoever greenlit this should never be in charge of the light " \
          "ever again. How dare they do this to me?!? Please don't go see this movie. And if you do, may God have " \
          "mercy on your soul. "


### 3. First prediction
Let's see if we can get probabilities for a review's sentiment.

In [ ]:
classifier = KerasClassifier()
probabs = classifier.get_probabilites(strings=[review_cats])
print(probabs)


### 4. First LIME explanation
We got almost 73% probability that this is a negative review. We can of course confirm that manually,
but let's let LIME explain why our model thinks that the review was negative.


In [ ]:
explainer = LimeTextExplainer(class_names=['positive', 'negative'])
explanation = explainer.explain_instance(review_cats, classifier.get_probabilites, num_features=6)
explanation.show_in_notebook()



### 4b. Another LIME explanation
Let's also look at a positive review:


In [ ]:
explanation2 = explainer.explain_instance(review_joker, classifier.get_probabilites, num_features=6)
explanation2.show_in_notebook()


